In [1]:
import atoti as tt

Welcome to atoti 0.7.1!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, set the ATOTI_DISABLE_TELEMETRY environment variable to True.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


In [2]:
session = tt.Session()

In [3]:
oulad_studentInfo = session.read_csv("data/studentInfo.csv",keys=["id_student"])

In [4]:
oulad_studentReg = session.read_csv("data/studentRegistration.csv",keys=["id_student"])
oulad_studentAss = session.read_csv("data/studentAssessment.csv",keys=["id_student"])
#oulad_studentVle = session.read_csv("data/studentVle.csv",keys=["id_student"])
oulad_assessment = session.read_csv("data/assessments.csv",keys=["id_assessment"])
#oulad_courses = session.read_csv("data/courses.csv",keys=["id_student"])
#oulad_vle = session.read_csv("data/vle.csv",keys=["id_site"])

In [18]:
oulad_studentReg.join(oulad_studentInfo)
oulad_studentAss.join(oulad_studentInfo)

In [21]:
ouladGender = oulad_studentInfo['gender']